# EAS: Post-procesamiento

Hasta ahora tenemos como resultado de la predicción de los modelos un directorio con imágenes del tamaño de los chips, que representan la probabilidad de que un píxel sea parte del objeto de interés (en este caso, panel fotovoltaico).

En este notebook se describen los pasos extras de post-procesamiento sobre estas imágenes resultado del modelo, para llegar a obtener un único archivo vectorial con polígonos de paneles.

In [1]:
import os

In [2]:
DATA_DIR = "data/paneles"
REMESA_DIR = os.path.join(DATA_DIR, 'predict', 'remesa5')

## Poligonizado

La siguiente función aplica una rutina de poligonización sobre los resultados de la predicción del modelo y genera un archivo vectorial en formato GeoPackage (GPKG). La rutina aplica `gdal_polygonize.py` a cada chip resultado generando un GPKG para cada chip, y luego une todos estos archivos en uno solo, de manera eficiente.

Antes de unirlos también aplica un umbral sobre los valores de los rásteres, que en este caso representan la probabilidad (valores entre 0 y 1).

In [3]:
from meduy.postprocess.polygonize import polygonize

In [4]:
# modelo 1: urbano
polygonize(threshold=0.4,
           input_dir=os.path.join(REMESA_DIR, 'urban', '400_400', 'results'),
           output=os.path.join(REMESA_DIR, 'urban', 'results_raw.gpkg'))

100%|██████████| 4/4 [00:02<00:00,  1.69it/s]


In [ ]:
# modelo 2: nacional
polygonize(threshold=0.4,
           input_dir=os.path.join(REMESA_DIR, 'national', '160_160', 'results'),
           output=os.path.join(REMESA_DIR, 'national', 'results_raw.gpkg'))

 12%|█▏        | 22235/184590 [13:44<1:52:27, 24.06it/s]

## Disolver polígonos

El segundo paso es la disolución de polígonos contiguos. El paso anterior de poligonización puede generar múltiples polígonos en forma de "pixel", aún si son contiguos (porque tienen valores de probabilidad distintos). Entonces esta rutina termina disolviendo todos esos polígonos contiguos en uno solo.

La función toma el GPKG generado en el paso anterior y devuelve uno nuevo.

In [ ]:
from meduy.postprocess.dissolve import dissolve

In [ ]:
# modelo 1: urbano
dissolve(src=os.path.join(REMESA_DIR, 'urban', 'results_raw.gpkg'),
         dst=os.path.join(REMESA_DIR, 'urban', 'results_diss.gpkg'))

In [ ]:
# modelo 2: nacional
dissolve(src=os.path.join(REMESA_DIR, 'urban', 'results_raw.gpkg'),
         dst=os.path.join(REMESA_DIR, 'urban', 'results_diss.gpkg'))

## Filtrar por área mínima

Como último paso, filtramos aquellos polígonos de area inferior a 50 m², dado que consideramos que son falsos positivos. Para esto utilizamos `ogr2ogr` y una consulta SQL.

In [ ]:
def filter_min_area(*, src, dst, min_area):
    import subprocess
    
    name, _ = os.path.splitext(os.path.basename(src))
    cmd = 'ogr2ogr ' \
        '-f "GPKG" ' \
        f'-sql "SELECT * FROM {name} m WHERE ST_Area(geom) > {min_area}" ' \
        '-dialect SQLITE ' \
        '-nln results ' \
        f'{dst} {src}'
    subprocess.run(cmd, shell=True, check=True)

In [ ]:
# modelo 1: urbano
filter_min_area(src=os.path.join(REMESA_DIR, 'urban', 'results_diss.gpkg'),
                dst=os.path.join(REMESA_DIR, 'urban', 'results_m50.gpkg'),
                min_area=50)

In [ ]:
# modelo 2: nacional
filter_min_area(src=os.path.join(REMESA_DIR, 'national', 'results_diss.gpkg'),
                dst=os.path.join(REMESA_DIR, 'national', 'results_m50.gpkg'),
                min_area=50)